# Start Simulation

In [1]:
import random
import numpy as np
import os
import time
from PIL import Image

import torch
torch.__version__
device = torch.device("cuda")

In [2]:
import getpass
user = getpass.getuser()
print(user)

yizhou


In [3]:
usd_path = f"omniverse://localhost/Users/{user}/uva_wall.usd"

In [4]:
from omni.isaac.kit import SimulationApp    
simulation_app = SimulationApp({"headless": True, "open_usd": usd_path,  "livesync_usd": usd_path}) 

Starting kit application with the fillowing args:  ['/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/exts/omni.isaac.kit/omni/isaac/kit/simulation_app.py', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/apps/omni.isaac.sim.python.kit', '--/app/tokens/exe-path=/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/kit', '--/persistent/app/viewport/displayOptions=3094', '--/rtx/materialDb/syncLoads=True', '--/rtx/hydra/materialSyncLoads=True--/omni.kit.plugin/syncUsdLoads=True', '--/app/renderer/resolution/width=1280', '--/app/renderer/resolution/height=720', '--/app/window/width=1440', '--/app/window/height=900', '--/renderer/multiGpu/enabled=True', '--ext-folder', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/exts', '--ext-folder', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/apps', '--/physics/cudaDevice=0', '--portable', '--no-window']
Passing the following args to the base kit application:  ['-f', '/home/yizhou/.local/share/jupyter/runtime/kernel-b7c05f67-f0ed-

OmniAssetFileFormat


Warp initialized:
   Version: 0.2.2
   CUDA device: NVIDIA GeForce RTX 3090
   Kernel cache: /home/yizhou/.cache/warp/0.2.2
[1.971s] [ext: omni.kit.menu.create-1.0.2] startup
[1.973s] [ext: omni.mdl-0.1.0] startup
[1.993s] [ext: omni.kit.window.file_importer-1.0.4] startup
[1.993s] [ext: omni.kit.window.file_exporter-1.0.4] startup
[1.995s] [ext: omni.kit.material.library-1.3.10] startup
[1.998s] [ext: omni.kit.window.drop_support-1.0.0] startup
[1.998s] [ext: omni.kit.window.file-1.3.16] startup
[2.000s] [ext: omni.kit.context_menu-1.3.9] startup
[2.004s] [ext: omni.kit.window.property-1.6.3] startup
[2.005s] [ext: omni.kit.window.content_browser-2.4.28] startup
[2.017s] [ext: omni.kit.widget.stage-2.6.15] startup
[2.021s] [ext: omni.isaac.version-1.0.0] startup
[2.021s] [ext: omni.kit.property.usd-3.14.9] startup
[2.056s] [ext: omni.kit.viewport.legacy_gizmos-1.0.0] startup
[2.060s] [ext: omni.hydra.rtx-0.1.0] startup
[2.070s] [ext: omni.renderer-rtx-0.0.0] startup
[2.071s] [ext: omn

In [5]:
# set log level
import logging
import carb

logging.getLogger("omni.hydra").setLevel(logging.ERROR)
logging.getLogger("omni.isaac.urdf").setLevel(logging.ERROR)
logging.getLogger("omni.physx.plugin").setLevel(logging.ERROR)

logging.getLogger("omni.isaac.synthetic_utils").setLevel(logging.ERROR)
logging.getLogger("omni.isaac.synthetic_utils.syntheticdata").setLevel(logging.ERROR)
logging.getLogger("omni.hydra.scene_delegate.plugin").setLevel(logging.ERROR)


l = carb.logging.LEVEL_ERROR
carb.settings.get_settings().set("/log/level", l)
carb.settings.get_settings().set("/log/fileLogLevel", l)
carb.settings.get_settings().set("/log/outputStreamLevel", l)

# # This logged error is printed as it should
# carb.log_error("ERROR")
# # This warning is printed but should not
# carb.log_warn("WARNING")

# Config

In [6]:
from task.config import DATA_PATH, FEATURE_PATH
task_type = "Wall"
side_choice = "Border"
base_asset_id = 0
load_nucleus = True

In [7]:
pause

NameError: name 'pause' is not defined

# Init Env

In [8]:
from uva_env import UvaEnv
env = UvaEnv()

In [9]:
from task.utils import add_scene_default
add_scene_default()

In [12]:
print(list(env.stage.TraverseAll()))

[Usd.Prim(</physicsScene>), Usd.Prim(</World>), Usd.Prim(</World/defaultLight>), Usd.Prim(</World/groundPlane>), Usd.Prim(</World/groundPlane/CollisionMesh>), Usd.Prim(</World/groundPlane/CollisionPlane>), Usd.Prim(</World/physicsScene>), Usd.Prim(</World/groundPlane_01>), Usd.Prim(</World/groundPlane_01/CollisionMesh>), Usd.Prim(</World/groundPlane_01/CollisionPlane>), Usd.Prim(</OmniverseKit_Persp>), Usd.Prim(</OmniverseKit_Front>), Usd.Prim(</OmniverseKit_Top>), Usd.Prim(</OmniverseKit_Right>), Usd.Prim(</Render>), Usd.Prim(</Render/RenderProduct_Viewport>), Usd.Prim(</Render/Vars>), Usd.Prim(</Render/Vars/LdrColor>)]


In [11]:
env.clean()
env.world.step(render=True)

# Scene

In [13]:
from task.scene import ArrangeScene
scene = ArrangeScene(task_type, side_choice, base_asset_id = 0, traj_id = 0, load_nucleus = load_nucleus)
env.scene = scene

In [14]:
# add base
# scene.add_base_asset()

In [15]:
# add room
scene.add_room()

asset_prim: omniverse://localhost/Users/yizhou/Asset/S/0/layout.usd


In [16]:
env.world.step(render=True)

In [17]:
# randomize scene
from layout.randomizer import Randomizer
randomizer = Randomizer()
randomizer.randomize_house(rand = True)

result:  Base
result:  vMaterials_2


In [18]:
env.world.step(render=True)

# Reward

In [19]:
from uv.reward import Rewarder

rewarder = Rewarder(env.world)
env.rewarder = rewarder

In [20]:
rewarder = Rewarder(env.world)
env.rewarder = rewarder

# Render

In [ ]:
from render.helper import RenderHelper
render = RenderHelper(task_type, side_choice)

render.add_task_cameras()
render.set_cameras()

xform_mat:  ( (-0.9999616399999998, 0, 0, 0), (0, 0.000019180000000118547, 0.9999808199999999, 0), (0, 0.9999808199999999, 0.000019180000000118547, 0), (0, 175, 125, 1) )
1.2] startup
[7.669s] [ext: omni.graph.window.core-1.23.4] startup
[7.675s] [ext: omni.graph.instancing-1.1.4] startup
[7.683s] [ext: omni.kit.menu.edit-1.0.6] startup
[7.686s] [ext: omni.isaac.isaac_sensor-1.0.2] startup
2022-12-27 22:11:47 [7,689ms] [Warning] [omni.physx.plugin] Deprecated: getSimulationEventStream is deprecated, please use getSimulationEventStreamV2
[7.700s] [ext: omni.graph.tutorials-1.1.2] startup
[7.717s] [ext: omni.graph.window.action-1.3.8] startup
[7.718s] [ext: omni.kit.widget.live-0.1.0] startup
[7.723s] [ext: omni.kit.widget.layers-1.5.17] startup
[7.737s] [ext: omni.graph.bundle.action-1.0.0] startup
[7.737s] [ext: omni.isaac.range_sensor-0.4.3] startup
[7.764s] [ext: omni.kit.property.layer-1.1.2] startup
[7.766s] [ext: omni.replicator.isaac-1.3.2] startup
[7.848s] [ext: omni.isaac.kit-0

2022-12-27 22:12:52 [73,331ms] [Error] [carb.graphics-vulkan.plugin] VkResult: ERROR_DEVICE_LOST
2022-12-27 22:12:52 [73,331ms] [Error] [carb.graphics-vulkan.plugin] vkWaitForFences failed for command queue.
2022-12-27 22:12:52 [73,331ms] [Error] [omni.kit.renderer.plugin] A GPU crash occurred. Exiting the application...
Reasons for the failure: a device lost, out of memory, or an unexpected bug.


[76.338s] [ext: omni.isaac.sim.python-2022.1.1] shutdown
[76.338s] [ext: omni.graph.visualization.nodes-1.1.1] shutdown
[76.364s] [ext: omni.replicator.isaac-1.3.2] shutdown
[76.364s] [ext: omni.graph.bundle.action-1.0.0] shutdown
[76.364s] [ext: omni.graph.window.action-1.3.8] shutdown
[76.392s] [ext: omni.graph.window.generic-1.3.8] shutdown
[76.430s] [ext: omni.graph.window.core-1.23.4] shutdown
[76.456s] [ext: omni.isaac.wheeled_robots-0.5.8] shutdown
[76.456s] [ext: omni.isaac.utils-0.1.11] shutdown
[76.544s] [ext: omni.isaac.isaac_sensor-1.0.2] shutdown
[76.640s] [ext: omni.isaac.core_nodes-0.13.0] shutdown
[76.642s] [ext: omni.isaac.range_sensor-0.4.3] shutdown
[76.663s] [ext: omni.isaac.dofbot-0.3.0] shutdown
[76.663s] [ext: omni.isaac.universal_robots-0.3.0] shutdown
[76.663s] [ext: omni.isaac.franka-0.3.0] shutdown
[76.663s] [ext: omni.isaac.motion_generation-3.6.1] shutdown
[76.663s] [ext: omni.isaac.manipulators-1.0.1] shutdown
[76.663s] [ext: omni.isaac.motion_planning-0.2

2022-12-27 22:12:57 [77,975ms] [Error] [carb.graphics-vulkan.plugin] VkResult: ERROR_DEVICE_LOST
2022-12-27 22:12:57 [77,975ms] [Error] [carb.graphics-vulkan.plugin] submitToQueueCommon failed.
2022-12-27 22:12:57 [77,976ms] [Error] [carb.graphics-vulkan.plugin] VkResult: ERROR_DEVICE_LOST
2022-12-27 22:12:57 [77,976ms] [Error] [carb.graphics-vulkan.plugin] submitToQueueCommon failed.
2022-12-27 22:12:57 [77,977ms] [Error] [carb.graphics-vulkan.plugin] VkResult: ERROR_DEVICE_LOST
2022-12-27 22:12:57 [77,977ms] [Error] [carb.graphics-vulkan.plugin] submitToQueueCommon failed.


18s] [ext: omni.kit.menu.edit-1.0.6] shutdown
[77.721s] [ext: omni.command.usd-1.0.1] shutdown
[77.721s] [ext: omni.kit.window.extensions-1.1.0] shutdown
[77.722s] [ext: semantics.schema.editor-0.2.3] shutdown
[77.823s] [ext: omni.kit.primitive.mesh-1.0.0] shutdown
[77.825s] [ext: omni.rtx.settings.core-0.5.5] shutdown
[77.827s] [ext: omni.rtx.window.settings-0.6.1] shutdown
[77.828s] [ext: omni.kit.window.preferences-1.2.1] shutdown
2022-12-27 22:12:57 [77,843ms] [Warning] [omni.ext.impl._internal] omni.kit.window.preferences-1.2.1 -> <class 'omni.kit.window.preferences.scripts.preferences_window.PreferencesExtension'>: extension object is still alive, something holds a reference on it. References: ["[0]:type: <class 'frame'>, id: 140275825571040", "[1]:type: <class 'frame'>, id: 140275128117680", "[2]:type: <class 'frame'>, id: 215983920"]
[77.850s] [ext: omni.kit.property.audio-1.0.5] shutdown
[77.850s] [ext: omni.kit.property.layer-1.1.2] shutdown
[77.850s] [ext: omni.kit.property.

2022-12-27 22:12:57 [78,200ms] [Error] [carb.graphics-vulkan.plugin] VkResult: ERROR_DEVICE_LOST
2022-12-27 22:12:57 [78,200ms] [Error] [carb.graphics-vulkan.plugin] vkWaitForFences failed for command queue.
2022-12-27 22:12:57 [78,200ms] [Error] [carb.graphics-vulkan.plugin] VkResult: ERROR_DEVICE_LOST
2022-12-27 22:12:57 [78,200ms] [Error] [carb.graphics-vulkan.plugin] vkWaitForFences failed for command queue.
2022-12-27 22:12:57 [78,200ms] [Error] [carb.graphics-vulkan.plugin] VkResult: ERROR_DEVICE_LOST
2022-12-27 22:12:57 [78,200ms] [Error] [carb.graphics-vulkan.plugin] vkWaitForFences failed for command queue.


[80.746s] [ext: omni.kit.notification_manager-1.0.5] shutdown
[80.746s] [ext: omni.kit.widget.browser_bar-2.0.3] shutdown
[80.746s] [ext: omni.kit.widget.zoombar-1.0.3] shutdown
[80.769s] [ext: omni.kit.window.script_editor-1.6.2] shutdown
[80.770s] [ext: omni.kit.widget.graph-1.4.3] shutdown
[80.770s] [ext: omni.kit.window.status_bar-0.1.1] shutdown
[80.770s] [ext: omni.kit.profiler.window-1.4.4] shutdown
[80.790s] [ext: omni.kit.uiapp-0.0.0] shutdown
[80.790s] [ext: omni.kit.mainwindow-0.0.0] shutdown
[80.790s] [ext: omni.kit.window.property-1.6.3] shutdown
[80.790s] [ext: omni.kit.widget.settings-1.0.0] shutdown
[80.790s] [ext: omni.kit.widget.versioning-1.3.8] shutdown
[80.790s] [ext: omni.kit.window.toolbar-1.2.4] shutdown
[80.791s] [ext: omni.kit.context_menu-1.3.9] shutdown
[80.791s] [ext: omni.kit.menu.create-1.0.2] shutdown
[80.792s] [ext: omni.kit.material.library-1.3.10] shutdown
2022-12-27 22:13:00 [80,785ms] [Warning] [omni.kit.menu.utils.scripts.utils] omni.kit.menu.utils

2022-12-27 22:13:00 [80,819ms] [Error] [carb.graphics-vulkan.plugin] VkResult: ERROR_DEVICE_LOST
2022-12-27 22:13:00 [80,819ms] [Error] [carb.graphics-vulkan.plugin] submitToQueueCommon failed.
2022-12-27 22:13:00 [80,819ms] [Error] [omni.kit.renderer.plugin] A GPU crash occurred. Exiting the application...
Reasons for the failure: a device lost, out of memory, or an unexpected bug.


[83.826s] [ext: omni.kit.renderer.core-0.0.0] shutdown
[83.826s] [ext: omni.appwindow-1.0.0] shutdown
[83.827s] [ext: omni.kit.loop-isaac-0.1.0] shutdown
[83.827s] [ext: omni.kit.pip_torch-1_11_0-0.1.3] shutdown
[83.884s] [ext: omni.usd.schema.isaac-0.2.0] shutdown
[83.885s] [ext: omni.physx.tensors-0.1.0] shutdown
[83.886s] [ext: omni.physx-1.4.15-5.1] shutdown
[83.905s] [ext: omni.convexdecomposition-1.4.15] shutdown
[83.922s] [ext: omni.localcache-0.0.0] shutdown
[83.938s] [ext: omni.kvdb-0.0.0] shutdown
[83.954s] [ext: omni.physics.tensors-0.1.0] shutdown
[83.970s] [ext: omni.usd.schema.forcefield-0.0.0] shutdown
[83.986s] [ext: omni.usdphysics-1.4.15] shutdown
[83.986s] [ext: omni.usd.schema.physx-0.0.0] shutdown
[84.003s] [ext: omni.debugdraw-0.1.0] shutdown
[84.003s] [ext: omni.hydra.engine.stats-1.0.0] shutdown
[84.003s] [ext: omni.graph.core-2.29.1] shutdown
[84.003s] [ext: omni.kit.stage_templates-1.1.2] shutdown
[84.003s] [ext: omni.kit.viewport.legacy_gizmos-1.0.0] shutdown

2022-12-27 22:13:03 [84,015ms] [Error] [carb.graphics-vulkan.plugin] VkResult: ERROR_DEVICE_LOST
2022-12-27 22:13:03 [84,015ms] [Error] [carb.graphics-vulkan.plugin] submitToQueueCommon failed.
2022-12-27 22:13:03 [84,029ms] [Error] [carb.graphics-vulkan.plugin] VkResult: ERROR_DEVICE_LOST
2022-12-27 22:13:03 [84,029ms] [Error] [carb.graphics-vulkan.plugin] submitToQueueCommon failed.


In [ ]:
pause

#  Learning

In [ ]:
# replay buffer
import json
from learning.replay_buffer import ReplayBuffer
from learning.config import *

buffer = ReplayBuffer(max_size=2000)

In [ ]:
# from learning.network.resnet import ResNetFeatureExtractor

# feature extraction
from learning.utils import extract_image_clip_feature_and_save, obtain_action_from_trainer
from transformers import CLIPProcessor, CLIPModel

feature_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
feature_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:
# trainer
from learning.network.sac import *

policy = Policy()

qf1 = QFunction()
qf2 = QFunction()
target_qf1 = QFunction()
target_qf2 = QFunction()

from learning.sac_trainer import SACTrainer

trainer = SACTrainer(policy, qf1, qf2, target_qf1, target_qf2, 
     use_automatic_entropy_tuning = False, 
     policy_lr=1e-3, 
     qf_lr=1e-3,
     target_update_period = 5)

# Trajectory

In [ ]:
use_network = True
debug = True

# Sampling

In [ ]:
total_traj = 0
total_step = 0

---------------------

In [ ]:
# traj config
for traj_id in range(1):
    total_traj += 1
    
    # constant
    base_asset_id = 0
    env.scene.base_asset_id = base_asset_id
    env.scene.traj_id = traj_id
    image_folder = os.path.join(DATA_PATH, task_type, side_choice, str(traj_id))

    # base
    # scene.add_base_asset()
    env.world.step(render = True)

    # get images
    env.world.render()
    images = render.get_images()
    render.save_rgb(images[0]['rgb'], image_folder, "0")
    
    ## extract feature
    if use_network:
        extract_image_clip_feature_and_save(f"{image_folder}/{0}.png", feature_model, feature_processor, 
            f"{image_folder}/{0}.pt",)
    

    # trajectory
    for step in range(3):
        total_step += 1
        
        # sample an object
        env.add_scene_obj(mode = "random")
        
        # TODO: get action from sampling
        if not use_network or total_traj < 10 or np.random.rand() < 0.2:
            x, y = np.tanh(np.random.randn()), np.tanh(np.random.randn())
        else:
            image_feature_file = f"{image_folder}/{step}.pt"
            
            object_info = env.scene.objects[-1]
            object_type = object_info["type"]
            obj_name = object_info["name"][:-4]
            object_feature_file = os.path.join(FEATURE_PATH, object_type, obj_name + ".pt")
            x, y = obtain_action_from_trainer(image_feature_file, object_feature_file, trainer, 
                                              scaler=np.exp(- total_traj / 100))
            object_info["use_network"] = True
        
        # load the object into the scene
        env.put_last_object((x, y)) 
        env.world.step(render=True)
        
        # register the object to the world for physics update
        env.register_last_object()
        env.world.step(render=True)

        # get images
        env.world.render()
        env.world.render()
        env.world.step(render=True)
        env.world.step(render=True)
        images = render.get_images()
        render.save_rgb(images[0]['rgb'], image_folder, str(step + 1))

        ## calculate reward
        env.calculate_last_reward(simulation_step = 30)
        
        ## extract feature
        if use_network:
            extract_image_clip_feature_and_save(f"{image_folder}/{step + 1}.png", feature_model, feature_processor, 
                f"{image_folder}/{step + 1}.pt",)
    

        ## reset
        env.world.reset()
        env.world.step(render=True)
        
        ## trainer nework
        if use_network and total_step % UPDATE_TRAINER_STEPS == 0 and total_traj > 5:
            batch = buffer.sample_batch(batch_size = BATCH_SIZE)
            trainer.update(batch)
            
            if debug and total_step % 10 == 0:
                rewards = batch['rewards'].to(device)
                terminals = batch['terminals'].to(device)
                obs = batch['observations'].to(device)
                actions = batch['actions'].to(device)
                next_obs = batch['next_observations'].to(device)
                obj_features = batch['object_features'].to(device)
                
                dist = trainer.policy(obs, obj_features)
                pred = trainer.qf1(obs, obj_features, actions)
                print(f"debug {total_traj}/{total_step}", #"\n dist: ", dist.mean.flatten().tolist(), dist.stddev.flatten().tolist(),
                      "\n pred:", pred.flatten().tolist(),
                      "\n rewards: ", rewards.flatten().tolist())


    # Record
    record = env.scene.get_scene_data()
    env.scene.save_scene_data()
    # print("record: ", record)
    
    # Add record to buffer
    if use_network:
        buffer.add_scene_sample(record)

    # Reset (env clean)
    env.clean(clean_all = False)
    env.step(render = True)

# Debug

In [ ]:
# get images
env.world.render()
env.world.render()
images = render.get_images()

Image.fromarray(images[0]['rgb'], "RGBA").resize((300, 300))

## Debug Reward

In [ ]:
traj_id = 0
# constant
base_asset_id = 0
env.scene.base_asset_id = base_asset_id
env.scene.traj_id = traj_id
image_folder = os.path.join(DATA_PATH, task_type, side_choice, str(traj_id))

# base
# scene.add_base_asset()
env.world.step(render = True)

# get images
env.world.render()
images = render.get_images()
render.save_rgb(images[0]['rgb'], image_folder, "0")

In [ ]:
# sample an object
env.add_scene_obj(mode = "random")

x, y = np.tanh(np.random.randn()), np.tanh(np.random.randn())

# load the object into the scene
env.put_last_object((x, y)) 
env.world.step(render=True)

# register the object to the world for physics update
env.register_last_object()
env.world.step(render=True)

# get images
env.world.render()
env.world.render()
env.world.step(render=True)
env.world.step(render=True)
images = render.get_images()
# render.save_rgb(images[0]['rgb'], image_folder, str(step + 1))

## calculate reward
env.calculate_last_reward(simulation_step = 30)

In [ ]:
env.get_last_object_box()

In [ ]:
env.scene.objects[-1]

In [ ]:
rewarder.reward_collision(env.scene.objects)

##  Debug Training

In [ ]:
import json
from learning.replay_buffer import ReplayBuffer

buffer = ReplayBuffer(max_size=1000)

for i in range(100):
    replay = json.load(open(f"{DATA_PATH}/{task_type}/{side_choice}/{i}/scene.json"))
    buffer.add_scene_sample(replay)

In [ ]:
len(buffer.dataset)

In [ ]:
# trainer
from learning.network.sac import *

policy = Policy()

qf1 = QFunction()
qf2 = QFunction()

target_qf1 = QFunction()
target_qf1.eval()

target_qf2 = QFunction()
target_qf2.eval()

from learning.sac_trainer import SACTrainer

trainer = SACTrainer(policy, qf1, qf2, target_qf1, target_qf2, 
     use_automatic_entropy_tuning = True, 
     policy_lr=1e-3, 
     qf_lr=1e-3,
     target_update_period = 5)

In [ ]:
self = trainer

In [ ]:
for _ in range(500):
    batch = buffer.sample_batch(batch_size=6)

    rewards = batch['rewards'].to(device)
    terminals = batch['terminals'].to(device)
    obs = batch['observations'].to(device)
    actions = batch['actions'].to(device)
    next_obs = batch['next_observations'].to(device)

    obj_features = batch['object_features'].to(device)

    # print("rewards", rewards)

    self.update(batch)
    
    if _ % 10 == 0:
        dist = self.policy(obs, obj_features)
        pred = self.qf1(obs, obj_features, actions)
        print(_, "\n dist   : ", [round(x, 3) for x in dist.mean.flatten().tolist()], 
                  [round(x, 3) for x in dist.stddev.flatten().tolist()],
              "\n pred   :", [round(x, 3) for x in pred.flatten().tolist()],
              "\n rewards: ", [round(x, 3) for x in rewards.flatten().tolist()])
    
#     # debug
#     pred = self.qf1(obs, obj_features, actions)
#     loss = self.qf_criterion(pred, rewards)
    
#     self.qf1_optimizer.zero_grad()
#     loss.backward()
#     self.qf1_optimizer.step()
    
#     if _ % 20 == 0:
#         print(_, "\n debug loss: ", loss.item(), "\n pred:", 
#               pred.flatten().tolist(), "\n rewards: ", rewards.flatten().tolist())